# OpinionChunk Embedding Pipeline (Titan + Neo4j)

## 1. Purpose

This pipeline embeds the **`OpinionChunk`** nodes in your Neo4j knowledge graph using **Amazon Titan** embeddings, and stores the resulting vectors back on each node.  

Once embeddings are stored, you can:

- Create a **vector index** in Neo4j.
- Use these embeddings for **semantic search**, **GraphRAG**, and **agent tools**.

---

## 2. What the Pipeline Does (Step by Step)

### 2.1. Load configuration and connect to Neo4j

- Reads environment variables from `.env`:
  - `NEO4J_URI`
  - `NEO4J_USERNAME`
  - `NEO4J_PASSWORD`
  - `NEO4J_DATABASE`
  - `AWS_REGION` (for Bedrock)
- Creates a Neo4j driver and verifies connectivity.

**Goal:** Make sure we can read and write `OpinionChunk` nodes.

---

### 2.2. Select `OpinionChunk` nodes that need embeddings

The pipeline runs a Cypher query roughly like:

```cypher
MATCH (c:OpinionChunk)
WHERE c.embedding IS NULL
  AND c.text IS NOT NULL AND c.text <> ''
RETURN id(c) AS node_id, c.text AS text
````

* It **filters only nodes without an `embedding` property** (so we do not recompute embeddings every time).
* It also ensures `text` is present and not empty.
* This returns a list of `(node_id, text)` pairs.

> Neo4j warnings:
>
> * `id(c)` is deprecated in Neo4j 5 → you can later switch to `elementId(c)` if you want.
> * `c.embedding` “does not exist” at first → this is only a warning because the property is not created yet. It will appear once the first embeddings are written.

---

### 2.3. Batch processing

The list of `OpinionChunk` nodes is processed in **batches** (for example, 32 at a time):

* Split the list of nodes into batches.
* For each batch:

  * Collect the chunk texts.
  * Call the Titan embedding function on these texts.
  * Write the resulting vectors back to Neo4j for the corresponding nodes.

This batching helps you:

* Control the number of Titan calls.
* Manage rate limits and memory usage more easily.

---

### 2.4. Embedding text with Amazon Titan

The helper function (conceptually called `embed_texts_titan`) does:

1. **Loop over each text** in the batch.

2. For each text, call **Bedrock InvokeModel** with:

   * `modelId = "amazon.titan-embed-text-v2:0"`
   * JSON body like:

     ```json
     { "inputText": "the text to embed" }
     ```
   * Titan expects a **single string**, not an array. This is why we call Titan once per chunk instead of sending a list.

3. Parse the response and extract the embedding vector from the response payload (e.g. `payload["embedding"]`).

4. Return a list of embeddings (one per input text).

**Key details:**

* The Titan model returns a vector of a fixed length (for v2 it is typically **1024 dimensions**, unless you override via `embeddingConfig`).
* These vectors must match the dimensions used in your Neo4j vector index.

---

### 2.5. Writing embeddings back to Neo4j

For each batch, after we get the embeddings list:

* We pair `node_id` (or `elementId`) with its embedding vector.
* We run a Cypher write query to set:

```cypher
MATCH (c:OpinionChunk)
WHERE id(c) = $id                      -- or elementId(c) = $id
SET c.embedding = $embedding
```

This adds a new property `embedding` to each `OpinionChunk` node.

After this step, some `OpinionChunk` nodes will have:

* `text` (original opinion snippet).
* `embedding` (Titan vector representation of that text).

---

### 2.6. “Force” flag logic

The pipeline supports a `force` parameter:

* `force = False` (default):

  * Only embed nodes where `c.embedding IS NULL`.
  * Good for incremental runs (e.g., new data, resume after a failure).
* `force = True`:

  * Ignore existing embeddings and re-embed everything.
  * Useful when you change the model, the embedding config, or want a clean refresh.

---

### 2.7. Creating the Neo4j vector index

After embeddings are written, the pipeline (or a separate step) creates a **vector index** on `OpinionChunk.embedding`, for example:

```cypher
CREATE VECTOR INDEX chunkEmbeddings IF NOT EXISTS
FOR (c:OpinionChunk)
ON (c.embedding)
OPTIONS {
  indexConfig: {
    `vector.dimensions`: 1024,
    `vector.similarity_function`: 'cosine'
  }
};
```

**Important:**
The `vector.dimensions` value must match the actual embedding size returned by Titan (e.g., 1024). If you use the wrong dimension, vector queries will fail.

This index is later used by retrievers / GraphRAG to:

* Run **semantic similarity search** over `OpinionChunk.text`.
* Retrieve the most relevant chunks for a user query.

---

## 3. How This Fits into the Larger System

Once this pipeline has run:

* Every relevant `OpinionChunk` node is enriched with an `embedding`.
* `chunkEmbeddings` vector index is available in Neo4j.
* You can now:

  * Use **vector-based retrievers** (e.g., in Python, LangChain, or Neo4j procedures).
  * Combine vector search with **Cypher traversal** (Vector + Cypher retrievers).
  * Plug this into a **GraphRAG agent** that:

    * Queries Neo4j by similarity.
    * Navigates relationships (cases, citations, parties, entities).
    * Uses an LLM (e.g., Claude) to generate answers.

In short:
This pipeline is the **embedding + indexing foundation** that makes semantic retrieval over `OpinionChunk` opinions possible.

---

## 4. Key Design Choices

* **Node label:** `:OpinionChunk`

  * Each node represents a chunk of legal opinion text.
* **Embedding property:** `embedding`

  * Stored directly on the node to keep graph + vector in one place.
* **Text property:** `text`

  * Source text used for embedding and later used as context for LLMs.
* **Model:** `amazon.titan-embed-text-v2:0` via Bedrock

  * Chosen to stay inside AWS (no OpenAI dependency).
* **Index name:** `chunkEmbeddings`

  * Used later by retrievers / Cypher vector queries.

---

## 5. Summary

This pipeline:

1. Finds all `OpinionChunk` nodes that need embeddings.
2. Calls Amazon Titan for each chunk to generate an embedding vector.
3. Writes the embedding back to Neo4j on `OpinionChunk.embedding`.
4. Creates a `chunkEmbeddings` vector index for fast similarity search.
5. Prepares your legal-knowledge graph for **GraphRAG**, **retrievers**, and **agents** built on top of `OpinionChunk` semantic search.


In [1]:
! pip install neo4j python-dotenv boto3

In [2]:
import os
import json
from typing import List, Dict

from neo4j import GraphDatabase
from dotenv import load_dotenv
import boto3
from math import ceil

# Load environment variables

load_dotenv("../.env", override=True)

NEO4J_URI      = os.getenv("NEO4J_URI")
NEO4J_USER     = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE") or None  # None = default

AWS_REGION     = os.getenv("AWS_REGION", "us-west-2")  # adjust to your setup

TITAN_MODEL_ID = "amazon.titan-embed-text-v2:0"
EMBEDDING_DIM  = 1024  # adjust if you configure Titan differently
INDEX_NAME     = "chunkEmbeddings"
LABEL_NAME     = "OpinionChunk"
EMBED_PROP     = "embedding"
MODEL_META     = "amazon.titan-embed-text-v2:0"

# Neo4j driver

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USER, NEO4J_PASSWORD),
)

# Optional: verify connectivity
driver.verify_connectivity()
print("Connected to Neo4j:", NEO4J_URI)

Connected to Neo4j: neo4j+s://df1a98d5.databases.neo4j.io


## Bedrock client and embedding helper

In [3]:
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)

def embed_texts_titan(texts):
    """
    Embed a list of strings using amazon.titan-embed-text-v2:0.
    Titan expects a single string in 'inputText', so we call it once per text.
    Returns: list[list[float]]
    """
    embeddings = []

    for t in texts:
        # Basic guard
        if not t or not t.strip():
            embeddings.append([])
            continue

        body = {
            "inputText": t,
            # Optional: if you want fixed dimension and your model supports it:
            # "embeddingConfig": {"outputEmbeddingLength": EMBEDDING_DIM}
        }

        response = bedrock.invoke_model(
            modelId=TITAN_MODEL_ID,
            contentType="application/json",
            accept="application/json",
            body=json.dumps(body),
        )

        payload = json.loads(response["body"].read())

        # Titan v2 typically returns an 'embedding' field
        if "embedding" in payload:
            vec = payload["embedding"]
        # Some wrappers use 'embeddings' for multiple, keep this as a fallback
        elif "embeddings" in payload:
            vec = payload["embeddings"][0]
        else:
            raise ValueError(f"Unexpected Titan response structure: {payload}")

        embeddings.append(vec)

    return embeddings


## Helper to fetch OpinionChunk nodes needing embeddings

In [4]:
def fetch_opinion_chunks(force: bool = False, limit: int | None = None):
    """
    Returns a list of dictionaries:
    {"node_id": <neo4j_internal_id>, "text": <chunk_text>}
    """
    if force:
        cypher = f"""
        MATCH (c:{LABEL_NAME})
        WHERE c.text IS NOT NULL AND c.text <> ''
        RETURN elementId(c) AS node_id, c.text AS text
        """
    else:
        cypher = f"""
        MATCH (c:{LABEL_NAME})
        WHERE c.{EMBED_PROP} IS NULL
          AND c.text IS NOT NULL AND c.text <> ''
        RETURN elementId(c) AS node_id, c.text AS text
        """

    if limit is not None:
        cypher += "\nLIMIT $limit"

    with driver.session(database=NEO4J_DATABASE) as session:
        result = session.run(cypher, limit=limit)
        rows = [{"node_id": r["node_id"], "text": r["text"]} for r in result]
    return rows


## Write embeddings back

In [5]:
def write_embeddings(batch: List[Dict]):
    """
    batch: list of dicts with keys: id (int), embedding (list[float])
    """
    cypher = f"""
    UNWIND $rows AS row
    MATCH (c)
    WHERE elementId(c) = row.id
    SET c.{EMBED_PROP}       = row.embedding,
        c.embedding_model    = $model_id,
        c.embedding_dim      = $dim,
        c.embedding_updated_at = datetime()
    """

    with driver.session(database=NEO4J_DATABASE) as session:
        session.run(
            cypher,
            rows=batch,
            model_id=MODEL_META,
            dim=EMBEDDING_DIM,
        )


## Main embedding pipeline

In [6]:
def run_embedding_pipeline(force: bool = False, batch_size: int = 32, limit: int | None = None):
    """
    force=False  -> only embed OpinionChunk nodes without embeddings.
    force=True   -> re-embed ALL OpinionChunk nodes.
    limit        -> optional limit for testing.
    """
    chunks = fetch_opinion_chunks(force=force, limit=limit)
    total = len(chunks)
    if total == 0:
        print("No OpinionChunk nodes to embed.")
        return

    print(f"Found {total} OpinionChunk nodes to embed (force={force}).")

    num_batches = ceil(total / batch_size)
    for i in range(num_batches):
        start = i * batch_size
        end   = min(start + batch_size, total)
        batch = chunks[start:end]

        texts = [row["text"] for row in batch]
        node_ids = [row["node_id"] for row in batch]

        print(f"Batch {i+1}/{num_batches} – embedding {len(texts)} chunks...")

        embeddings = embed_texts_titan(texts)

        rows_for_neo4j = [
            {"id": node_ids[j], "embedding": embeddings[j]}
            for j in range(len(node_ids))
        ]

        write_embeddings(rows_for_neo4j)

    print("Embedding pipeline completed.")


## Create vector index on :OpinionChunk(embedding)

In [7]:
def create_vector_index():
    cypher = f"""
    CREATE VECTOR INDEX {INDEX_NAME} IF NOT EXISTS
    FOR (c:{LABEL_NAME}) ON (c.{EMBED_PROP})
    OPTIONS {{
      indexConfig: {{
        `vector.dimensions`: $dim,
        `vector.similarity_function`: 'cosine'
      }}
    }}
    """
    with driver.session(database=NEO4J_DATABASE) as session:
        session.run(cypher, dim=EMBEDDING_DIM)
    print(f"Vector index '{INDEX_NAME}' ensured on :{LABEL_NAME}({EMBED_PROP}) with dim={EMBEDDING_DIM}.")

## Run Embeddings on OpinionChunks

In [8]:
run_embedding_pipeline(force=False, batch_size=32)
create_vector_index()

Received notification from DBMS server: <GqlStatusObject gql_status='01N52', status_description='warn: property key does not exist. The property `embedding` does not exist in database `neo4j`. Verify that the spelling is correct.', position=<SummaryInputPosition line=3, column=17, offset=48>, raw_classification='UNRECOGNIZED', classification=<NotificationClassification.UNRECOGNIZED: 'UNRECOGNIZED'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'UNRECOGNIZED', '_severity': 'WARNING', '_position': {'offset': 48, 'line': 3, 'column': 17}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "\n        MATCH (c:OpinionChunk)\n        WHERE c.embedding IS NULL\n          AND c.text IS NOT NULL AND c.text <> ''\n        RETURN elementId(c) AS node_id, c.text AS text\n        "


Found 56347 OpinionChunk nodes to embed (force=False).
Batch 1/1761 – embedding 32 chunks...
Batch 2/1761 – embedding 32 chunks...
Batch 3/1761 – embedding 32 chunks...
Batch 4/1761 – embedding 32 chunks...
Batch 5/1761 – embedding 32 chunks...
Batch 6/1761 – embedding 32 chunks...
Batch 7/1761 – embedding 32 chunks...
Batch 8/1761 – embedding 32 chunks...
Batch 9/1761 – embedding 32 chunks...
Batch 10/1761 – embedding 32 chunks...
Batch 11/1761 – embedding 32 chunks...
Batch 12/1761 – embedding 32 chunks...
Batch 13/1761 – embedding 32 chunks...
Batch 14/1761 – embedding 32 chunks...
Batch 15/1761 – embedding 32 chunks...
Batch 16/1761 – embedding 32 chunks...
Batch 17/1761 – embedding 32 chunks...
Batch 18/1761 – embedding 32 chunks...
Batch 19/1761 – embedding 32 chunks...
Batch 20/1761 – embedding 32 chunks...
Batch 21/1761 – embedding 32 chunks...
Batch 22/1761 – embedding 32 chunks...
Batch 23/1761 – embedding 32 chunks...
Batch 24/1761 – embedding 32 chunks...
Batch 25/1761 – em